In [1]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Helper Clases / Functions

In [2]:
def Load_Data(num):
    if (num == -1): # All data
        X_all = []
        y_all = []
        for i in range(8):
            file_path = './../project_datasets/A0' + str(i+1) + 'T_slice.mat'
            data = h5py.File(file_path, 'r')
            X = np.copy(data['image'])
            y = np.copy(data['type'])
            X = X[:, 0:23, :]
            X_all.append(X)
            y = y[0,0:X.shape[0]:1]
            y_all.append(y)
        A, N, E, T = np.shape(X_all)
        X_all = np.reshape(X_all, (A*N, E, T))
        y_all = np.reshape(y_all, (-1))
        y_all = y_all - 769
        ## Remove NAN
        index_Nan = []
        for i in range(A*N):
            for j in range(E):
                if (any(np.isnan(X_all[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X_all = np.delete(X_all, index_Nan, axis=0)
        y_all = np.delete(y_all, index_Nan)
        return (X_all, y_all)
    else:
        file_path = './../project_datasets/A0' + str(num) + 'T_slice.mat'
        data = h5py.File(file_path, 'r')
        X = np.copy(data['image'])
        y = np.copy(data['type'])
        X = X[:, 0:23, :]
        y = y[0,0:X.shape[0]:1]
        y = y - 769
         ## Remove NAN
        N, E, T = np.shape(X)
        index_Nan = []
        for i in range(N):
            for j in range(E):
                if (any(np.isnan(X[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X = np.delete(X, index_Nan, axis=0)
        y = np.delete(y, index_Nan)
        return (X, y)

# Load Data

In [3]:
X, y = Load_Data(-1) # -1 to load all datas
N, E, T = np.shape(X)
print (np.shape(X))

(2280, 23, 1000)


# Make DataLoaders

In [4]:
bs_train = 200
bs_val = 100
bs_test = 100
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dataloaders = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-bs_val-bs_test, bs_val, bs_test])
dataloaders['train'] = data_utils.DataLoader(dset['train'], batch_size=bs_train, shuffle=True, num_workers=1)
dataloaders['val'] = data_utils.DataLoader(dset['val'], batch_size=bs_val, shuffle=True, num_workers=1)
dataloaders['test'] = data_utils.DataLoader(dset['test'], batch_size=bs_test, shuffle=True, num_workers=1)

# Define Model

In [13]:
class myLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myLSTM, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.conv_temp = nn.Conv2d(1,40,tuple([1,25]))
        self.conv_elec = nn.Conv3d(1,23,tuple([40, 23, 1]))
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layer, dropout=0.5)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        x.unsqueeze_(1)
        out_conv_temp = self.conv_temp(x)
        out_conv_temp = out_conv_temp.unsqueeze_(1)
        out_conv_elec = self.conv_elec(out_conv_temp)
        out_conv_elec_sque= torch.squeeze(out_conv_elec)
        out_swap = out_conv_elec_sque.permute(2,0,1)
        out_lstm, _ = self.lstm(out_swap)
        out_lstm = out_lstm[-1, :, :]
        out = self.classifier(out_lstm)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

In [14]:
dtype = torch.cuda.FloatTensor
hidden_dim = 20
num_classes = 4
num_epoches = 10
model = myLSTM(E, hidden_dim, 1, num_classes)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Conv Training

In [15]:
best_acc = 0.35

In [16]:
for epoch in range(num_epoches):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        X_train, y_train = Variable(X_train), Variable(y_train)
        # forward + backward + optimize
        out = model(X_train.cuda())
        # print (out)
        loss = loss_fn(out, y_train.long().cuda())
        print('(%d batch) loss: %f' % (i, loss))
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = model.check_accuracy(dataloaders['train'])
    val_acc = model.check_accuracy(dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epoches, train_acc, val_acc))
    if (val_acc > best_acc):
        best_acc = val_acc
        torch.save(model, 'best_LSTM_CONV_DO.pt')

(0 batch) loss: 1.415361
(1 batch) loss: 1.401772
(2 batch) loss: 1.407655
(3 batch) loss: 1.404101
(4 batch) loss: 1.393789
(5 batch) loss: 1.397655
(6 batch) loss: 1.419183
(7 batch) loss: 1.401364
(8 batch) loss: 1.418651
(9 batch) loss: 1.405664
(10 batch) loss: 1.426728
(Epoch 1 / 10) train_acc: 0.297115; val_acc: 0.220000
(0 batch) loss: 1.375411
(1 batch) loss: 1.396444
(2 batch) loss: 1.377945
(3 batch) loss: 1.376789
(4 batch) loss: 1.372616
(5 batch) loss: 1.383889
(6 batch) loss: 1.351731
(7 batch) loss: 1.384928
(8 batch) loss: 1.381230
(9 batch) loss: 1.412637
(10 batch) loss: 1.336806
(Epoch 2 / 10) train_acc: 0.318750; val_acc: 0.240000
(0 batch) loss: 1.364723
(1 batch) loss: 1.377169
(2 batch) loss: 1.360615
(3 batch) loss: 1.364594
(4 batch) loss: 1.350826
(5 batch) loss: 1.365352
(6 batch) loss: 1.356017
(7 batch) loss: 1.347816
(8 batch) loss: 1.366390
(9 batch) loss: 1.390154
(10 batch) loss: 1.349258
(Epoch 3 / 10) train_acc: 0.357212; val_acc: 0.230000
(0 batch) 

# Best Model Test

In [10]:
best_model = torch.load('best_LSTM_CONV.pt')
print (best_model.check_accuracy(dataloaders['val']))

0.35


/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
